In [1]:
from datetime import date
import pandas as pd
import sys
sys.path.append('../src')
import data.clean as clean
import numpy as np

# Read external data into a pandas df

In [2]:
#today = date.today()
#d = today.strftime('%Y_%m_%d')
d = '2020_11_25'

df = pd.read_csv(f'../data/external/raw_chewy_data_{d}.csv')

# Clean data

## Unpack attribute and guarenteed analysis (GA) dictionaries
Attribute information and guarenteed analysis (GA) information are stored as dictionaries in their columns. This is because different products have different fields/numbers of fields in these tables. So first things first, expand those two columns.

In [3]:
# help(clean.dict_col_to_cols)

df = clean.dict_col_to_cols(df=df, col='ga_dict', ga=True)
df = clean.dict_col_to_cols(df=df, col='attr_dict', ga=False)

## Fill guarenteed analysis columns missing critical statistics

It is mandatory that cat food manufacturers report % protein min, % fat min, and % fiber max in their guarenteed analysis. So it's likely that when there isn't a value for one of these macros, but there is a value for a very similarly named macro, a typo has been made. So we need to fill empty macro values with the values most likely to be correct.

In [4]:
df['% protein min'] = df['% protein min'].fillna(df['% protein']).fillna(df['% protein max'])
df['% fat min'] = df['% fat min'].fillna(df['% fat']).fillna(df['% fat max'])
df['% fiber max'] = df['% fiber max'].fillna(df['% fiber']).fillna(df['% fiber min'])
df['% moisture max'] = df['% moisture max'].fillna(df['% moisture']).fillna(df['% moisture min'])

# df = df.dropna(thresh = df.shape[0] * 0.5, how = 'all', axis = 1)

## Add up percentages of misc. guarenteed analysis componenets 
There are so many miscellaneous compounds that are included in only a handful of the products' guarenteed analysis. We want the percent values for these compounds, because they will help us calculate % carbohydrates later, but we don't need them individually. So we want to add up the columns that start with '%' and are not protein, fat, fiber, moisture, or carbohydrate (a few products have this value, but we will be calculating it for all products). If the sum of these columns is 100% or greater, we know something has gone terribly wrong (I found a typo...) so we will set that value to nan.

In [5]:
macros = ('protein', 'fat', 'fiber', 'moisture', 'carbohydrate')

misc_components = [col for col in df if '%' in col and not any(macro in col for macro in macros)]

df['% misc components'] = df[misc_components].sum(axis=1)

df.loc[df['% misc components'] >= 100, '% misc components'] = np.nan

## Extract kilocalories/mass or kilocalories/volume
The calories column contains information about the number of kilocalories per unit mass or volume of food. However, the units are not consistent, and often multiple values are reported. So we can pull out the kcal/unit mass for dry food, and the kcal/unit volume for wet food (this is because total product size is given in mass for dry food and volume for wet food, and we'd like to eventually calculate U.S. dollar per kcal).

In [6]:
# help(clean.get_unit_val_col)

# kcal/unit of mass:
df = clean.get_unit_val_col(df, column_name = 'calories',
                  col_split_regex = '(?<!/) (?=\d)|,|;|:',
                  unit_name = 'kcal_per_kg',
                  unit_regex = 'kcal.*kg')

df = clean.get_unit_val_col(df, column_name = 'calories',
                  col_split_regex = '(?<!/) (?=\d)|,|;|:',
                  unit_name = 'kcal_per_lb',
                  unit_regex = 'kcal.*lb')

# kcal/unit of volume:
df = clean.get_unit_val_col(df, column_name = 'calories',
                  col_split_regex = '(?<!/) (?=\d)|,|;|:',
                  unit_name = 'kcal_per_oz',
                  unit_regex = 'kcal.*oz|kcal.*ounce')

df = clean.get_unit_val_col(df, column_name = 'calories',
                  col_split_regex = '(?<!/) (?=\d)|,|;|:',
                  unit_name = 'kcal_per_cup',
                  unit_regex = 'kcal.*cup')

# kcal/item (kcal in a single can, pouch, etc.)
df = clean.get_unit_val_col(df, column_name = 'calories',
                  col_split_regex = '(?<!/) (?=\d)|,|;|:',
                  unit_name = 'kcal_per_item',
                  unit_regex = 'kcal.*can|kcal.*pouch|kcl.*pouch|kcal.*unit|kcal.*bowl|kcal.*tray|kcal.*container|kcal.*serving|kcal.*tub|kcal.*pack',
                  clear_inconsistent_rows = False,
                  get_denom_col = True,
                  denom_name = 'item_volume_for_kcal',
                  denom_regex = '((?<=\/).*?\d+\.?\d+)')

## Extract item mass, volume, and/or count
For dry food we need the total mass of the bag, for wet food we need the total volume of the can/item unit and the number of cans/item units.

In [7]:
# number of lbs per item
df = clean.get_unit_val_col(df, column_name = 'name',
                  col_split_regex = ',',
                  unit_name = 'total_mass_lb',
                  unit_regex = 'lb|pound|Pound')

# if no number lbs found, get that value from the weight attribute 
# DO NOT DO THIS WEIGHTS ARE NOT ALL IN LBS
# df['total_mass_lb'] = df['total_mass_lb'].fillna(df['weight'])

# number of ounces per item
df = clean.get_unit_val_col(df, column_name = 'name',
                  col_split_regex = ',',
                  unit_name = 'item_oz',
                  unit_regex = 'oz|ounce')

# number of items
df = clean.get_unit_val_col(df, column_name = 'name',
                  col_split_regex = ',',
                  unit_name = 'item_count',
                  unit_regex = 'case of')

# if no number of items specified, set item_count to 1
df['item_count'] = df['item_count'].fillna(1)

# Filling values and adding warnings
## Nutritional info
For nutritional information, we need to know AT LEAST the % protein min, % fat min, and % moisture. % fiber max should also be provided. To calculate the cost per kilocalorie, we need the prouct price, and the kilocalories per product.

In [8]:
missing_ga = df[df['% protein min'].isna() | df['% fat min'].isna() | df['% fiber max'].isna()]

print(f'There are %d products out of %d which are missing protein, fat, or fiber info.' 
      % (len(missing_ga), len(df)))

missing_moisture = df[df['% protein min'].notnull() & df['% fat min'].notnull() & df['% fiber max'].notnull() & df['% moisture max'].isna()]

print(f'There are %d products out of %d which have all protein, fat, and fiber info, but not moisture.' 
      % (len(missing_moisture), len(df)))

There are 16 products out of 2095 which are missing protein, fat, or fiber info.
There are 54 products out of 2095 which have all protein, fat, and fiber info, but not moisture.


The average wet cat food and the average dry cat food are supposed to have 75-78% and 10-12% moisture max, respectively (https://www.petfoodinstitute.org/the-whole-bowl/z-pet-food-4-things-know-dry-pet-food/).
We could assign those moisture values to the foods that are missing moisture information but have all the other nutrient values.

Out of curiosity, do those values reflect the average moisture for wet and dry cat food in this data?

In [9]:
wet_mean = df[df['food form'].str.contains('wet food', na=False)]['% moisture max'].mean()

dry_mean = df[df['food form'].str.contains('dry food', na=False)]['% moisture max'].mean()

print(f'In this data, wet food has a mean percent moisture max of %d percent, \
and dry food has a mean percent moisture max of %d percent.' 
      % (wet_mean, dry_mean))

In this data, wet food has a mean percent moisture max of 80 percent, and dry food has a mean percent moisture max of 9 percent.


### Assuming moisture content
For all products that have values for protein, fat, and fiber, but are missing a value for moisture, we can fill in the moisture value with an estimate based on if the food is wet or dry. But if we do this, we should add a warning column stating that this value was estimated.

In [10]:
df['moisture_warning'] = np.nan

df.loc[df['food form'].str.contains('wet food') 
       & df['% protein min'].notnull() 
       & df['% fat min'].notnull() 
       & df['% fiber max'].notnull() 
       & df['% moisture max'].isna(), 
       ['% moisture max', 'moisture_warning']] = [76.5, '% moisture max estimated']

df.loc[df['food form'].str.contains('dry food') 
       & df['% protein min'].notnull() 
       & df['% fat min'].notnull() 
       & df['% fiber max'].notnull() 
       & df['% moisture max'].isna(), 
       ['% moisture max', 'moisture_warning']] = [11, '% moisture max estimated']

## Kilocalories/product
Calculating kilocalories per product is a bit trickier. We need to make sure we're not mistaking mass with volume. When wet food is measured in ounces, they are fluid ounces. But if we're given the calorie content of a wet food only in kcal/kg, we can't convert fluid ounces to kilograms, because we don't know the density of the wet food (though most wet food is ~78% water).

So for each product we need price, and either:

1. kcal/unit of volume (cups or ounces)
2. item volume (cups or ounces)

OR

1. kcal/unit of mass (kg or lb)
2. item mass (kg or lb)

OR

1. kcal/item
2. item count

In [11]:
trouble = df[((df['kcal_per_cup'].isna() & df['kcal_per_oz'].isna()) | df['item_oz'].isna()) & 
             ((df['kcal_per_kg'].isna() & df['kcal_per_lb'].isna()) | df['total_mass_lb'].isna()) &
             df['kcal_per_item'].isna()]

print(f'There are %d products out of %d which are missing required kcal/product values.' 
      % (len(trouble), len(df)))

There are 212 products out of 2095 which are missing required kcal/product values.


Let's see how many of these we could work with if we were willing to convert between mass and volume...
We'll do calculations for kilocalories/product later, but for now we'll put a warning on these products.

In [12]:
annoying = trouble[(trouble['kcal_per_cup'].notnull() | trouble['kcal_per_oz'].notnull() | 
                    trouble['kcal_per_kg'].notnull() | trouble['kcal_per_lb'].notnull()) & 
                   (trouble['item_oz'].notnull() | trouble['total_mass_lb'].notnull())].copy()

print(f'We could salvage %d of the aforementioned %d products if we were willing to convert between mass and volume.' 
      % (len(annoying), len(trouble)))

df['conversion_warning'] = np.nan

df.loc[annoying.index, 'conversion_warning'] = 'kcal/product required a conversion between mass and volume'

We could salvage 146 of the aforementioned 212 products if we were willing to convert between mass and volume.


Good to know for later...

# Final clean up and save new tsv
Get rid of columns we don't need anymore. Save as tsv because I think they look neater than csvs.

In [13]:
desired_cols = ['name', 'url', 'our_price', 'list_price', 'ingredients', '% protein min', '% fat min', '% fiber max', 
                '% misc components', '% moisture max', 'item number','weight', 'packaging type', 'food texture', 
                'brand', 'lifestage', 'food form', 'special diet', 'breed size', 'dimensions', 'small pet type', 
                'restricted availability', 'kcal_per_kg', 'kcal_per_lb', 'kcal_per_oz', 'kcal_per_cup', 'kcal_per_item',
                'item_volume_for_kcal', 'total_mass_lb', 'item_oz', 'item_count', 'moisture_warning', 
                'conversion_warning']

final_cols = [col for col in df.columns if col in desired_cols]

df_lite = df[final_cols]

df_lite.columns = df_lite.columns.str.replace(' ', '_')

df_lite.fillna(np.nan)

df_lite.to_csv(f'../data/interim/cleaned_chewy_data_{d}.txt', sep='\t', index=False)